In [1]:
import pandas as pd
import json
import os
import re

In [2]:
pickle_files = [
    "./data/Sonnet.pkl",
    "./data/sonnet-3.5.pkl",
    "./data/gpt-4o.pkl",
    "./data/gemini-1.5-pro.pkl",
]

dfs = [pd.read_pickle(file) for file in pickle_files]
df = pd.concat(dfs)

df

,image_name,depth,center,initial_size,reduction_factor,line_thickness,padding,model_output_raw,extracted_number,is_correct,Model
0,nested_squares_depth_2_image_1_thickness_2.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,2,0.75,The image contains 2 squares. There is one out...,2,True,Sonnet
1,nested_squares_depth_2_image_1_thickness_3.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,3,0.75,The image contains 2 squares. There is an oute...,2,True,Sonnet
2,nested_squares_depth_2_image_1_thickness_4.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,4,0.75,"The image shows 2 squares, one larger square f...",2,True,Sonnet
3,nested_squares_depth_2_image_2_thickness_2.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,2,0.75,The image consists of two squares - a larger s...,2,True,Sonnet
4,nested_squares_depth_2_image_2_thickness_3.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,3,0.75,"In the provided image, I can count a total of ...",2,True,Sonnet
...,...,...,...,...,...,...,...,...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,3,0.75,There are **5** squares in the image. \n,5,True,Gemini-1.5-Pro
116,nested_squares_depth_5_image_9_thickness_4.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,4,0.75,There are **5** squares in the image. \n,5,True,Gemini-1.5-Pro
117,nested_squares_depth_5_image_10_thickness_2.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,2,0.75,There are **5** squares in the image. \n,5,True,Gemini-1.5-Pro
118,nested_squares_depth_5_image_10_thickness_3.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,3,0.75,There are **5** squares in the image. \n,5,True,Gemini-1.5-Pro


In [10]:
accuracy_per_model_and_word = df.groupby(["Model"])["is_correct"].mean()

# Convert the accuracy to percentage and print the result
accuracy_per_model_and_word = accuracy_per_model_and_word * 100
# round 2 decimal places
accuracy_per_model_and_word = accuracy_per_model_and_word.round(2)

accuracy_per_model_and_word.reset_index()

,Model,is_correct
0,Gemini-1.5-Pro,80.00
1,Sonnet,55.00
2,Sonnet-3.5,87.50
3,gpt-4o,48.33


In [4]:
accuracy_per_model_and_depth = df.groupby(["Model", "depth"])["is_correct"].mean()

# Convert the accuracy to percentage
accuracy_per_model_and_depth = accuracy_per_model_and_depth * 100
# Round to 2 decimal places
accuracy_per_model_and_depth = accuracy_per_model_and_depth.round(2)

print(accuracy_per_model_and_depth)

Model           depth
Gemini-1.5-Pro  2        100.00
                3        100.00
                4         20.00
                5        100.00
Sonnet          2         80.00
                3         23.33
                4         63.33
                5         53.33
Sonnet-3.5      2        100.00
                3        100.00
                4        100.00
                5         50.00
gpt-4o          2         76.67
                3         40.00
                4         20.00
                5         56.67
Name: is_correct, dtype: float64


In [5]:
print(accuracy_per_model_and_depth.to_latex())

\begin{tabular}{llr}
\toprule
 &  & is_correct \\
Model & depth &  \\
\midrule
\multirow[t]{4}{*}{Gemini-1.5-Pro} & 2 & 100.000000 \\
 & 3 & 100.000000 \\
 & 4 & 20.000000 \\
 & 5 & 100.000000 \\
\cline{1-3}
\multirow[t]{4}{*}{Sonnet} & 2 & 80.000000 \\
 & 3 & 23.330000 \\
 & 4 & 63.330000 \\
 & 5 & 53.330000 \\
\cline{1-3}
\multirow[t]{4}{*}{Sonnet-3.5} & 2 & 100.000000 \\
 & 3 & 100.000000 \\
 & 4 & 100.000000 \\
 & 5 & 50.000000 \\
\cline{1-3}
\multirow[t]{4}{*}{gpt-4o} & 2 & 76.670000 \\
 & 3 & 40.000000 \\
 & 4 & 20.000000 \\
 & 5 & 56.670000 \\
\cline{1-3}
\bottomrule
\end{tabular}



In [8]:
accuracy_per_model_and_depth = df.groupby(["Model", "line_thickness"])[
    "is_correct"
].mean()

# Convert the accuracy to percentage
accuracy_per_model_and_depth = accuracy_per_model_and_depth * 100
# Round to 2 decimal places
accuracy_per_model_and_depth = accuracy_per_model_and_depth.round(2)

accuracy_per_model_and_depth

Model           line_thickness
Gemini-1.5-Pro  2                 80.0
                3                 80.0
                4                 80.0
Sonnet          2                 42.5
                3                 60.0
                4                 62.5
Sonnet-3.5      2                 87.5
                3                 87.5
                4                 87.5
gpt-4o          2                 52.5
                3                 45.0
                4                 47.5
Name: is_correct, dtype: float64

In [7]:
accuracy_per_model_and_depth.reset_index()

,Model,line_thickness,is_correct
0,Gemini-1.5-Pro,2,80.0
1,Gemini-1.5-Pro,3,80.0
2,Gemini-1.5-Pro,4,80.0
3,Sonnet,2,42.5
4,Sonnet,3,60.0
5,Sonnet,4,62.5
6,Sonnet-3.5,2,87.5
7,Sonnet-3.5,3,87.5
8,Sonnet-3.5,4,87.5
9,gpt-4o,2,52.5
